<a href="https://colab.research.google.com/github/SingTown/openmv_tensorflow_training_scripts/blob/main/mnist/openmv_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train mnist and Save to OpenMV

This Code is for TensorFlow 2

reference: https://keras.io/examples/vision/mnist_convnet/

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Prepare Dataset

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# Define Model

In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0


# Train model

In [4]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 44s 101ms/step - loss: 0.3753 - accuracy: 0.8859 - val_loss: 0.0829 - val_accuracy: 0.9772
Epoch 2/15
422/422 [==============================] - 35s 82ms/step - loss: 0.1114 - accuracy: 0.9663 - val_loss: 0.0582 - val_accuracy: 0.9837
Epoch 3/15
422/422 [==============================] - 36s 85ms/step - loss: 0.0843 - accuracy: 0.9735 - val_loss: 0.0554 - val_accuracy: 0.9855
Epoch 4/15
422/422 [==============================] - 35s 82ms/step - loss: 0.0718 - accuracy: 0.9776 - val_loss: 0.0441 - val_accuracy: 0.9867
Epoch 5/15
422/422 [==============================] - 35s 82ms/step - loss: 0.0623 - accuracy: 0.9808 - val_loss: 0.0391 - val_accuracy: 0.9897
Epoch 6/15
422/422 [==============================] - 34s 82ms/step - loss: 0.0559 - accuracy: 0.9824 - val_loss: 0.0364 - val_accuracy: 0.9900
Epoch 7/15
422/422 [==============================] - 35s 83ms/step - loss: 0.0531 - accuracy: 0.9832 - val_loss: 0.0366 - val_accuracy

# Evaluate the trained model

In [5]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.025147348642349243
Test accuracy: 0.991599977016449


# Full Integer Quantization
ref: https://www.tensorflow.org/lite/performance/post_training_quantization#full_integer_quantization

In [6]:
import tensorflow as tf
from tensorflow import lite

def representative_dataset():
    for i in range(100):
        yield [np.array([x_test[i]])]

# Convert the tflite.
converter = lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_quant_model = converter.convert()

# Save the model.
with open('trained.tflite', 'wb') as f:
  f.write(tflite_quant_model)

/usr/local/lib/python3.9/dist-packages/tensorflow/lite/python/convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


# Succeed
Copy trained.tflite to OpenMV4 H7 or OpenMV4 H7 Plus, run this code in OpenMV: https://github.com/SingTown/openmv_tensorflow_training_scripts/blob/main/mnist/main.py